# # OSRS Grand Exchange Data Fetcher
# Fetches historical price data and publishes as a Kaggle dataset

In [1]:
import requests
import pandas as pd
from datetime import datetime
import os
import json

In [2]:
import requests
import pandas as pd

def get_item_name(item_id):
    """Fetch item name from OSRS Wiki API (kept for filename generation)"""
    url = f"https://oldschool.runescape.wiki/w/Special:Lookup?type=item&id={item_id}"
    try:
        response = requests.get(url, timeout=5, headers={'User-Agent': 'OSRS Data Collector'})
        if response.status_code == 200:
            return response.url.split('/')[-1].replace('_', ' ').split('#')[0].strip()
        return f"Item_{item_id}"
    except Exception as e:
        print(f"Error getting item name: {e}")
        return f"Item_{item_id}"

def prepare_dataset(item_id):
    """Generate dataset with only Date and Price (GP)"""
    # Fetch data (original unchanged)
    url = "https://api.weirdgloop.org/exchange/history/osrs/all"
    try:
        response = requests.get(url, params={"id": str(item_id)}, timeout=10)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None
    
    # Process data (now minimal)
    item_key = next(k for k in data.keys() if k.isdigit())
    df = pd.DataFrame(data[item_key])
    
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df[['Date', 'price']].rename(columns={'price': 'Price (GP)'})
    
    # Keep original filename format
    item_name = get_item_name(item_id)
    clean_name = (
        item_name.lower()
        .replace(' ', '_')
        .replace("'", "")
        .replace('(', '')
        .replace(')', '')
    )
    filename = f"osrs_{clean_name}_history.csv"
    
    # Save with original naming
    df.to_csv(filename, index=False)
    print(f"Saved dataset to {filename}")
    print(df)
    return df

# Example usage (bond ID = 13190):
bond_data = prepare_dataset(13190)

Saved dataset to osrs_old_school_bond_history.csv
                    Date  Price (GP)
0    2015-03-28 00:00:00     2000000
1    2015-03-29 00:00:00     2000000
2    2015-03-30 00:00:00     1902808
3    2015-03-31 00:00:00     1807667
4    2015-04-01 00:00:00     1717283
...                  ...         ...
3562 2025-04-03 17:19:46    15028304
3563 2025-04-04 12:50:28    14825123
3564 2025-04-05 17:37:06    14731840
3565 2025-04-06 14:48:01    14573630
3566 2025-04-07 06:37:12    14539620

[3567 rows x 2 columns]
